In [1]:
%pip install pandas

  Using cached pandas-2.3.0-cp311-cp311-win_amd64.whl (11.1 MB)
  Using cached numpy-2.3.1-cp311-cp311-win_amd64.whl (13.0 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cu128/torch-2.7.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu128/torch-2.7.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cu128/torchvision-0.22.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu128/torchvision-0.22.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cu128/torchaudio-2.7.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu128/torchaudio-2.7.1%2Bcu128-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for filelock from https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.me


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T

In [4]:
mnist_data_train = datasets.MNIST(root="./root", train=True, download=True, transform=T.ToTensor())
mnist_data_test = datasets.MNIST(root="./root", train=False, download=True, transform=T.ToTensor())

In [5]:
train_dataloader = DataLoader(mnist_data_train, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_dataloader = DataLoader(mnist_data_test, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(28*28, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 10)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        x = self.activation(x)
        x = self.layer3(x)
        x= self.activation(x)
        return x

In [7]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
device

device(type='cuda')

In [8]:
model = NeuralNetwork().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [9]:
def train(train_loader : DataLoader, model: nn.Module, criterion : nn.Module, optimizer : torch.optim):
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        

In [10]:
train(train_dataloader, model, criterion, optimizer)

In [11]:
torch.Tensor([1]).grad

In [12]:
np.array([1])

array([1])